## IMPORTS

In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.transforms import ToTensor
from pandas import Series
import numpy as np
from sklearn import preprocessing
import random
from datetime import datetime, timedelta
from warnings import simplefilter
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error,roc_auc_score
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# thius is my try for create data and analyze the real data

## analayzing the real data

In [ ]:
#The dataset, collected on the Marconi 100 system, has been published and
#described in the Scientific dataset publication: https://www.nature.com/articles/s41597-023-02174-3
#It is available on Zenodo: https://zenodo.org/records/7541722


#This is rack 1 of the dataset
!wget https://zenodo.org/record/7541722/files/1.tar?download=1 -o rack.tar

In [ ]:
!tar -xf /content/1.tar?download=1

In [ ]:
#This is node 36, of rack 1
file_name = '36.parquet'

In [ ]:
DATA = pd.read_parquet(file_name)
DATA.head(5)

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,2020-03-09 12:00:00+00:00,21.636362,0.077138,21.600000,21.799999,29.0,0.0,29.0,29.0,30.0,...,22.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
1,2020-03-09 12:15:00+00:00,21.860000,0.128063,21.600000,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399998,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
2,2020-03-09 12:30:00+00:00,21.885714,0.098975,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
3,2020-03-09 12:45:00+00:00,21.918917,0.098194,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,20.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
4,2020-03-09 13:00:00+00:00,21.875553,0.096967,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.400001,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>


In [ ]:
item_types=[]
item=[]
for i in range(len(DATA)):
    if not pd.isna(DATA['value'][i]):
        if DATA['value'][i] in item_types:
            continue
        else:
            item.append(DATA.iloc[i])
            item_types.append(DATA['value'][i])

In [ ]:
item_types   ### 0 is normal data and 2,3 is anomaly and they replace it with 1 to make it easy

[0, 2, 3]

In [ ]:
item

[timestamp          2020-05-11 14:45:00+00:00
 ambient_avg                        21.340543
 ambient_std                         0.091413
 ambient_min                        21.200001
 ambient_max                             21.4
                              ...            
 total_power_avg                   561.081081
 total_power_std                    52.183863
 total_power_min                        520.0
 total_power_max                        860.0
 value                                      0
 Name: 3331, Length: 354, dtype: object,
 timestamp          2020-06-11 14:15:00+00:00
 ambient_avg                             22.0
 ambient_std                              0.0
 ambient_min                             22.0
 ambient_max                             22.0
                              ...            
 total_power_avg                   402.666667
 total_power_std                     6.798693
 total_power_min                        400.0
 total_power_max                       

In [ ]:
# Combine the list of Series into a single DataFrame
df = pd.DataFrame(item)

# Function to split the DataFrame based on a pattern and include the column names and their values
def split_dataframe_by_pattern(df, pattern):
    columns = [col for col in df.columns if pattern in col]
    result = []

    for index, row in df.iterrows():
        row_data = {col: row[col] for col in columns}
        row_data['timestamp'] = row['timestamp']
        row_data['value'] = row['value']
        result.append(row_data)

    return pd.DataFrame(result)

In [ ]:
df_avg = split_dataframe_by_pattern(df, 'avg')
df_std = split_dataframe_by_pattern(df, 'std')
df_min = split_dataframe_by_pattern(df, 'min')
df_max = split_dataframe_by_pattern(df, 'max')

In [ ]:
def split_dataframe_columns(df, num_columns_per_chunk):
    columns = df.columns
    num_chunks = (len(columns) + num_columns_per_chunk - 1) // num_columns_per_chunk
    chunks = [columns[i * num_columns_per_chunk:(i + 1) * num_columns_per_chunk] for i in range(num_chunks)]
    return [df[chunk] for chunk in chunks]

# Define the number of columns per chunk
num_columns_per_chunk = 5

# Split the DataFrame into chunks
df_chunks = split_dataframe_columns(df_avg, num_columns_per_chunk)

In [ ]:
def display_df_in_chunks(df, num_columns_per_chunk):
    df_chunks = split_dataframe_columns(df, num_columns_per_chunk)
    for i, chunk in enumerate(df_chunks):
        print(f"Chunk {i + 1}:")
        print(chunk)
        print("\n")

In [ ]:
df1, df2, df3, df4, df5 = df_chunks

In [ ]:
df1

,ambient_avg,dimm0_temp_avg,dimm10_temp_avg,dimm11_temp_avg,dimm12_temp_avg,dimm13_temp_avg,dimm14_temp_avg,dimm15_temp_avg,dimm1_temp_avg,dimm2_temp_avg,dimm3_temp_avg,dimm4_temp_avg,dimm5_temp_avg,dimm6_temp_avg,dimm7_temp_avg,dimm8_temp_avg,dimm9_temp_avg,fan0_0_avg,fan0_1_avg,fan1_0_avg
0,21.340543,29.000000,30.0,29.0,30.0,31.0,29.027027,31.0,29.0,29.0,29.0,29.000000,29.000000,29.000000,29.0,29.0,30.000000,4305.405405,4697.297297,4300.0
1,22.000000,29.000000,30.0,30.0,30.0,31.0,29.511111,31.0,30.0,29.0,30.0,29.000000,29.000000,29.000000,29.0,29.0,30.000000,4313.333333,4697.777778,4300.0
2,22.311112,29.933333,30.0,30.0,31.0,32.0,30.000000,32.0,30.0,30.0,30.0,29.155556,29.888889,29.844444,29.0,29.0,30.733333,4324.444444,4700.000000,4300.0


In [ ]:
df2

,fan1_1_avg,fan2_0_avg,fan2_1_avg,fan3_0_avg,fan3_1_avg,fan_disk_power_avg,gpu0_core_temp_avg,gpu0_mem_temp_avg,gpu1_core_temp_avg,gpu1_mem_temp_avg,gpu3_core_temp_avg,gpu3_mem_temp_avg,gpu4_core_temp_avg,gpu4_mem_temp_avg,gv100card0_avg,gv100card1_avg,gv100card3_avg,gv100card4_avg,p0_core10_temp_avg,p0_core11_temp_avg
0,4700.0,4305.405405,4697.297297,4316.216216,4700.0,31.675676,41.0,36.243243,42.702703,39.324324,37.891892,34.27027,43.810811,40.513514,0.0,0.0,0.0,0.0,39.027027,38.972973
1,4700.0,4300.000000,4700.000000,4331.111111,4700.0,31.688889,36.0,33.000000,37.000000,35.000000,33.000000,31.00000,38.000000,36.000000,0.0,0.0,0.0,0.0,36.644444,36.644444
2,4700.0,4300.000000,4695.555556,4306.666667,4700.0,32.133333,37.0,33.000000,37.000000,35.844444,33.288889,31.00000,39.000000,37.000000,0.0,0.0,0.0,0.0,36.955556,36.977778


In [ ]:
df3

,p0_core12_temp_avg,p0_core13_temp_avg,p0_core14_temp_avg,p0_core15_temp_avg,p0_core16_temp_avg,p0_core17_temp_avg,p0_core18_temp_avg,p0_core19_temp_avg,p0_core20_temp_avg,p0_core21_temp_avg,p0_core22_temp_avg,p0_core23_temp_avg,p0_core8_temp_avg,p0_core9_temp_avg,p0_io_power_avg,p0_mem_power_avg,p0_power_avg,p0_vdd_temp_avg,p1_core10_temp_avg,p1_core11_temp_avg
0,39.000000,39.000000,39.000000,39.027027,39.000000,38.972973,39.000000,38.972973,38.972973,38.972973,38.972973,39.027027,50.000000,47.486486,53.945946,18.000000,43.351351,30.108108,38.702703,38.729730
1,36.644444,36.622222,36.622222,36.622222,36.577778,36.600000,36.555556,36.577778,36.577778,36.577778,36.555556,36.533333,36.533333,36.644444,53.577778,18.000000,22.000000,29.933333,35.644444,35.622222
2,36.955556,36.933333,36.933333,36.888889,36.955556,36.933333,36.933333,36.888889,36.844444,36.844444,36.844444,36.822222,36.933333,36.933333,53.755556,18.044444,22.000000,30.000000,35.711111,35.711111


In [ ]:
df4

,p1_core16_temp_avg,p1_core17_temp_avg,p1_core18_temp_avg,p1_core19_temp_avg,p1_core20_temp_avg,p1_core21_temp_avg,p1_core2_temp_avg,p1_core3_temp_avg,p1_core4_temp_avg,p1_core5_temp_avg,p1_core6_temp_avg,p1_core7_temp_avg,p1_core8_temp_avg,p1_core9_temp_avg,p1_io_power_avg,p1_mem_power_avg,p1_power_avg,p1_vdd_temp_avg,pcie_avg,ps0_input_power_avg
0,38.594595,38.594595,38.540541,38.513514,38.675676,38.540541,48.243243,49.621622,38.567568,38.621622,38.729730,38.621622,38.702703,38.756757,50.378378,18.000000,47.459459,30.837838,39.000000,317.027027
1,35.577778,35.733333,35.688889,35.711111,35.733333,35.733333,35.466667,35.555556,35.755556,35.711111,35.600000,35.577778,35.733333,35.733333,49.844444,18.000000,25.022222,29.000000,36.300005,230.666667
2,35.688889,35.688889,35.688889,35.711111,35.688889,35.711111,35.800000,35.733333,35.666667,35.622222,35.622222,35.711111,35.711111,35.777778,51.111111,18.177778,24.488889,29.533333,36.900003,230.222222


In [ ]:
df5

,ps0_input_voltag_avg,ps0_output_curre_avg,ps0_output_volta_avg,ps1_input_power_avg,ps1_input_voltag_avg,ps1_output_curre_avg,ps1_output_volta_avg,total_power_avg,timestamp,value
0,231.567568,22.108108,12.200001,355.945946,231.675676,24.000000,12.200001,561.081081,2020-05-11 14:45:00+00:00,0
1,232.000000,14.933333,12.400001,287.555556,232.488889,18.311111,12.400001,402.666667,2020-06-11 14:15:00+00:00,2
2,227.688889,14.644444,12.400001,283.333333,228.000000,18.377778,12.400001,402.222222,2021-09-28 09:30:00+00:00,3


In [ ]:
df_avg

,ambient_avg,dimm0_temp_avg,dimm10_temp_avg,dimm11_temp_avg,dimm12_temp_avg,dimm13_temp_avg,dimm14_temp_avg,dimm15_temp_avg,dimm1_temp_avg,dimm2_temp_avg,...,ps0_input_voltag_avg,ps0_output_curre_avg,ps0_output_volta_avg,ps1_input_power_avg,ps1_input_voltag_avg,ps1_output_curre_avg,ps1_output_volta_avg,total_power_avg,timestamp,value
0,21.340543,29.000000,30.0,29.0,30.0,31.0,29.027027,31.0,29.0,29.0,...,231.567568,22.108108,12.200001,355.945946,231.675676,24.000000,12.200001,561.081081,2020-05-11 14:45:00+00:00,0
1,22.000000,29.000000,30.0,30.0,30.0,31.0,29.511111,31.0,30.0,29.0,...,232.000000,14.933333,12.400001,287.555556,232.488889,18.311111,12.400001,402.666667,2020-06-11 14:15:00+00:00,2
2,22.311112,29.933333,30.0,30.0,31.0,32.0,30.000000,32.0,30.0,30.0,...,227.688889,14.644444,12.400001,283.333333,228.000000,18.377778,12.400001,402.222222,2021-09-28 09:30:00+00:00,3


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['ambient_avg'].plot(kind='hist', bins=20, title='ambient_avg')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['dimm0_temp_avg'].plot(kind='hist', bins=20, title='dimm0_temp_avg')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['dimm11_temp_avg'].plot(kind='hist', bins=20, title='dimm11_temp_avg')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['dimm10_temp_avg']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'dimm10_temp_avg'}, axis=1)
              .sort_values('dimm10_temp_avg', ascending=True))
  xs = counted['dimm10_temp_avg']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('dimm10_temp_avg', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('dimm10_temp_avg')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_12['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['ambient_avg'].plot(kind='line', figsize=(8, 4), title='ambient_avg')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['dimm0_temp_avg'].plot(kind='line', figsize=(8, 4), title='dimm0_temp_avg')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['dimm11_temp_avg'].plot(kind='line', figsize=(8, 4), title='dimm11_temp_avg')
plt.gca().spines[['top', 'right']].set_visible(False)

### get the data and seprate it until animaly disappear

In [ ]:
#The dataset, collected on the Marconi 100 system, has been published and
#described in the Scientific dataset publication: https://www.nature.com/articles/s41597-023-02174-3
#It is available on Zenodo: https://zenodo.org/records/7541722


#This is rack 1 of the dataset
!wget https://zenodo.org/record/7541722/files/1.tar?download=1 -o rack.tar

In [ ]:
!tar -xf /content/1.tar?download=1

In [ ]:
#This is node 36, of rack 1
file_name = '36.parquet'

In [ ]:
DATA = pd.read_parquet(file_name)
DATA.head(5)

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,2020-03-09 12:00:00+00:00,21.636362,0.077138,21.600000,21.799999,29.0,0.0,29.0,29.0,30.0,...,22.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
1,2020-03-09 12:15:00+00:00,21.860000,0.128063,21.600000,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399998,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
2,2020-03-09 12:30:00+00:00,21.885714,0.098975,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
3,2020-03-09 12:45:00+00:00,21.918917,0.098194,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,20.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
4,2020-03-09 13:00:00+00:00,21.875553,0.096967,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.400001,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>


In [ ]:
def get_segments(df):
    segments = []
    segment = []
    in_segment = False
    for i, row in df.iterrows():
        if pd.isna(row['value']) or row['value'] == 0 :
            if in_segment:
                segments.append(pd.DataFrame(segment))
                segment = []
                in_segment = False
        else:
            row['value']=1   ## test nashode va miam 2 ,3 ro be 1 tabdil mikonim ke rahat tar bashim
            in_segment = True
        segment.append(row)
    if segment:
        segments.append(pd.DataFrame(segment))
    return segments

In [ ]:
seprate_data =get_segments(DATA)

In [ ]:
dseprate_data

[                     timestamp  ambient_avg  ambient_std  ambient_min  \
 0    2020-03-09 12:00:00+00:00    21.636362     0.077138    21.600000   
 1    2020-03-09 12:15:00+00:00    21.860000     0.128063    21.600000   
 2    2020-03-09 12:30:00+00:00    21.885714     0.098975    21.799999   
 3    2020-03-09 12:45:00+00:00    21.918917     0.098194    21.799999   
 4    2020-03-09 13:00:00+00:00    21.875553     0.096967    21.799999   
 ...                        ...          ...          ...          ...   
 6307 2020-06-11 14:45:00+00:00    22.008889     0.041216    22.000000   
 6308 2020-06-11 15:00:00+00:00    22.128889     0.095737    22.000000   
 6309 2020-06-11 15:15:00+00:00    22.200001     0.000000    22.200001   
 6310 2020-06-11 15:30:00+00:00    22.177778     0.062854    22.000000   
 6311 2020-06-11 15:45:00+00:00    22.200001     0.000000    22.200001   
 
       ambient_max  dimm0_temp_avg  dimm0_temp_std  dimm0_temp_min  \
 0       21.799999            29.0      

In [ ]:
len(seprate_data)

142

In [ ]:
for item in seprate_data:
  print(len(item))

6312
744
978
835
3
808
6002
1162
110
1247
11
4683
2
5376
6170
4
3
3
3
3
3
3
4
8
6
3
2
5
3
4
8
3
3
4
3
3
11
6
4
4
7
3
6
8
2
4
9
3
4
2
6
5
7
4
5
2
4
4
4
12
4
2
6
6
2
4
12
2
2
4
2
4
4
4
2
7
6
4
2
3
4
3
2
5
3
9
3
5
2
15
3
4
2
5
4
4
4
3
2
6
3
3
7
2
3
2
2
3
2
4
3
1913
15
1904
775
5302
2658
4156
13
29
2232
480
2622
4698
1338
48
3333
1423
1325
97
668
3845
20
1829
1336
1342
204
582
926
1166
4786
729


In [ ]:
df_0=pd.DataFrame(seprate_data[0]) ### how get all of ? is it neccessary to get all of it?

In [ ]:
df_4=pd.DataFrame(seprate_data[4])

In [ ]:
df_4

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
8869,2020-07-08 07:15:00+00:00,25.400000,0.301109,24.600000,25.799999,31.977778,0.147406,31.0,32.0,33.444444,...,22.0,12.400001,0.0,12.4,12.4,419.111111,4.121608,400.0,420.0,0
8870,2020-07-08 07:30:00+00:00,25.537779,0.167715,25.200001,26.000000,32.000000,0.000000,32.0,32.0,33.044444,...,23.0,12.400001,0.0,12.4,12.4,419.555556,2.948111,400.0,420.0,2
8871,2020-07-08 07:45:00+00:00,24.995554,1.076198,22.200001,25.799999,31.533333,0.805536,30.0,32.0,32.888889,...,22.0,12.400001,0.0,12.4,12.4,417.777778,6.285394,400.0,420.0,2


In [ ]:
df_0

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,2020-03-09 12:00:00+00:00,21.636362,0.077138,21.600000,21.799999,29.0,0.0,29.0,29.0,30.0,...,22.0,12.399999,0.0,12.4,12.4,400.000000,0.000000,400.0,400.0,NaN
1,2020-03-09 12:15:00+00:00,21.860000,0.128063,21.600000,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399998,0.0,12.4,12.4,400.000000,0.000000,400.0,400.0,NaN
2,2020-03-09 12:30:00+00:00,21.885714,0.098975,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399999,0.0,12.4,12.4,400.000000,0.000000,400.0,400.0,NaN
3,2020-03-09 12:45:00+00:00,21.918917,0.098194,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,20.0,12.399999,0.0,12.4,12.4,400.000000,0.000000,400.0,400.0,NaN
4,2020-03-09 13:00:00+00:00,21.875553,0.096967,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.400001,0.0,12.4,12.4,400.000000,0.000000,400.0,400.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6307,2020-06-11 14:45:00+00:00,22.008889,0.041216,22.000000,22.200001,29.0,0.0,29.0,29.0,30.0,...,22.0,12.400001,0.0,12.4,12.4,404.888889,8.595146,400.0,420.0,2
6308,2020-06-11 15:00:00+00:00,22.128889,0.095737,22.000000,22.200001,29.0,0.0,29.0,29.0,30.0,...,22.0,12.400001,0.0,12.4,12.4,404.888889,8.595146,400.0,420.0,2
6309,2020-06-11 15:15:00+00:00,22.200001,0.000000,22.200001,22.200001,29.0,0.0,29.0,29.0,30.0,...,21.0,12.400001,0.0,12.4,12.4,403.111111,7.248670,400.0,420.0,2
6310,2020-06-11 15:30:00+00:00,22.177778,0.062854,22.000000,22.200001,29.0,0.0,29.0,29.0,30.0,...,22.0,12.400001,0.0,12.4,12.4,402.222222,6.285394,400.0,420.0,2


In [ ]:
# Combine the list of Series into a single DataFrame
# df = pd.DataFrame(DATA)

# Function to split the DataFrame based on a pattern and include the column names and their values
def split_dataframe_by_pattern(df, pattern):
    columns = [col for col in df.columns if pattern in col]
    result = []

    for index, row in df.iterrows():
        row_data = {col: row[col] for col in columns}
        row_data['timestamp'] = row['timestamp']
        row_data['value'] = row['value']
        result.append(row_data)

    return pd.DataFrame(result)

In [ ]:
df_avg = split_dataframe_by_pattern(df_0, 'avg')
df_std = split_dataframe_by_pattern(df_0, 'std')
df_min = split_dataframe_by_pattern(df_0, 'min')
df_max = split_dataframe_by_pattern(df_0, 'max')

In [ ]:
def split_dataframe_columns(df, num_columns_per_chunk):
    columns = df.columns
    num_chunks = (len(columns) + num_columns_per_chunk - 1) # num_columns_per_chunk
    chunks = [columns[i * num_columns_per_chunk:(i + 1) * num_columns_per_chunk] for i in range(num_chunks)]
    return [df[chunk] for chunk in chunks]

# Define the number of columns per chunk
num_columns_per_chunk = 15

# Split the DataFrame into chunks
df_chunks = split_dataframe_columns(df_avg, num_columns_per_chunk)

In [ ]:
#, df5,df6, df7, df8, df9, df10 ,df11, df12, df13, df14, df15 ,df16, df17
df1, df2, df3, df4 , df5,df6 = df_chunks

In [ ]:
df1

,ambient_avg,dimm0_temp_avg,dimm10_temp_avg,dimm11_temp_avg,dimm12_temp_avg,dimm13_temp_avg,dimm14_temp_avg,dimm15_temp_avg,dimm1_temp_avg,dimm2_temp_avg,dimm3_temp_avg,dimm4_temp_avg,dimm5_temp_avg,dimm6_temp_avg,dimm7_temp_avg
0,21.636362,29.0,30.0,29.151515,30.151515,31.0,29.606061,31.0,29.696970,29.000000,30.0,29.0,29.0,29.0,28.393939
1,21.860000,29.0,30.0,29.741935,30.733333,31.0,29.966667,31.0,30.000000,29.000000,30.0,29.0,29.0,29.0,28.966667
2,21.885714,29.0,30.0,30.000000,31.000000,31.0,30.000000,31.0,30.000000,29.000000,30.0,29.0,29.0,29.0,29.000000
3,21.918917,29.0,30.0,29.972973,31.000000,31.0,30.000000,31.0,30.000000,29.000000,30.0,29.0,29.0,29.0,29.000000
4,21.875553,29.0,30.0,29.955556,31.000000,31.0,30.000000,31.0,29.977778,29.000000,30.0,29.0,29.0,29.0,29.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6307,22.008889,29.0,30.0,30.000000,30.000000,31.0,30.000000,31.0,30.000000,29.000000,30.0,29.0,29.0,29.0,29.000000
6308,22.128889,29.0,30.0,30.000000,30.177778,31.0,30.000000,31.0,30.000000,29.000000,30.0,29.0,29.0,29.0,29.000000
6309,22.200001,29.0,30.0,30.000000,30.555556,31.0,30.000000,31.0,30.000000,29.000000,30.0,29.0,29.0,29.0,29.000000
6310,22.177778,29.0,30.0,30.000000,30.733333,31.0,30.000000,31.0,30.000000,29.066667,30.0,29.0,29.0,29.0,29.000000


In [ ]:
df2

,dimm8_temp_avg,dimm9_temp_avg,fan0_0_avg,fan0_1_avg,fan1_0_avg,fan1_1_avg,fan2_0_avg,fan2_1_avg,fan3_0_avg,fan3_1_avg,fan_disk_power_avg,gpu0_core_temp_avg,gpu0_mem_temp_avg,gpu1_core_temp_avg,gpu1_mem_temp_avg
0,29.0,30.0,4303.030303,4700.000000,4300.0,4700.0,4303.030303,4700.000000,4315.151515,4700.000000,32.484848,35.0,32.545455,37.000000,34.181818
1,29.0,30.0,4306.451613,4696.774194,4300.0,4700.0,4306.666667,4700.000000,4316.666667,4700.000000,32.600000,35.0,32.906250,37.000000,34.500000
2,29.0,30.0,4302.857143,4700.000000,4300.0,4700.0,4305.714286,4700.000000,4311.428571,4700.000000,32.400000,35.0,32.971429,37.114286,34.857143
3,29.0,30.0,4305.405405,4700.000000,4300.0,4700.0,4310.810811,4700.000000,4327.027027,4700.000000,32.378378,35.0,32.945946,37.270270,34.945946
4,29.0,30.0,4304.444444,4700.000000,4300.0,4700.0,4306.666667,4700.000000,4317.777778,4700.000000,32.088889,35.0,32.955556,37.288889,34.888889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6307,29.0,30.0,4313.333333,4700.000000,4300.0,4700.0,4300.000000,4700.000000,4331.111111,4700.000000,32.044444,36.0,33.000000,37.000000,35.000000
6308,29.0,30.0,4320.000000,4700.000000,4300.0,4700.0,4300.000000,4697.777778,4324.444444,4700.000000,31.866667,36.0,33.000000,37.000000,35.000000
6309,29.0,30.0,4317.777778,4700.000000,4300.0,4700.0,4300.000000,4700.000000,4326.666667,4697.777778,32.222222,36.0,33.000000,37.000000,35.000000
6310,29.0,30.0,4320.000000,4700.000000,4300.0,4700.0,4300.000000,4700.000000,4304.444444,4700.000000,32.088889,36.0,33.000000,37.000000,35.000000


In [ ]:
df3

,gpu3_core_temp_avg,gpu3_mem_temp_avg,gpu4_core_temp_avg,gpu4_mem_temp_avg,gv100card0_avg,gv100card1_avg,gv100card3_avg,gv100card4_avg,p0_core10_temp_avg,p0_core11_temp_avg,p0_core12_temp_avg,p0_core13_temp_avg,p0_core14_temp_avg,p0_core15_temp_avg,p0_core16_temp_avg
0,34.0,30.939394,37.0,35.090909,0.0,0.0,0.0,0.0,36.272727,36.242424,36.242424,36.212121,36.212121,36.242424,36.242424
1,34.0,31.000000,37.0,35.125000,0.0,0.0,0.0,0.0,36.290323,36.258065,36.258065,36.166667,36.266667,36.300000,36.266667
2,34.0,31.000000,37.0,35.342857,0.0,0.0,0.0,0.0,36.371429,36.371429,36.371429,36.371429,36.342857,36.342857,36.371429
3,34.0,31.000000,37.0,35.540541,0.0,0.0,0.0,0.0,36.459459,36.486486,36.540541,36.432432,36.432432,36.432432,36.459459
4,34.0,31.000000,37.0,35.622222,0.0,0.0,0.0,0.0,36.422222,36.444444,36.333333,36.355556,36.355556,36.377778,36.377778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6307,33.0,31.000000,38.0,36.000000,0.0,0.0,0.0,0.0,36.533333,36.533333,36.466667,36.466667,36.733333,36.711111,36.688889
6308,33.0,31.000000,38.0,36.044444,0.0,0.0,0.0,0.0,36.733333,36.755556,36.777778,36.733333,36.866667,36.866667,36.800000
6309,33.0,31.000000,38.0,36.111111,0.0,0.0,0.0,0.0,36.844444,36.844444,36.866667,36.866667,36.822222,36.822222,36.844444
6310,33.0,31.000000,38.0,36.311111,0.0,0.0,0.0,0.0,36.866667,36.844444,36.844444,36.844444,36.977778,36.955556,36.888889


In [ ]:
df4

,p0_core17_temp_avg,p0_core18_temp_avg,p0_core19_temp_avg,p0_core20_temp_avg,p0_core21_temp_avg,p0_core22_temp_avg,p0_core23_temp_avg,p0_core8_temp_avg,p0_core9_temp_avg,p0_io_power_avg,p0_mem_power_avg,p0_power_avg,p0_vdd_temp_avg,p1_core10_temp_avg,p1_core11_temp_avg
0,36.181818,36.181818,36.242424,36.272727,36.242424,36.181818,36.212121,36.212121,36.272727,53.333333,18.000000,21.939394,29.000000,35.272727,35.272727
1,36.250000,36.250000,36.166667,36.281250,36.281250,36.300000,36.366667,36.166667,36.281250,53.343750,18.000000,22.000000,29.218750,35.266667,35.437500
2,36.428571,36.428571,36.457143,36.342857,36.400000,36.428571,36.400000,36.371429,36.342857,53.457143,18.000000,22.000000,29.142857,35.428571,35.428571
3,36.486486,36.486486,36.432432,36.459459,36.459459,36.459459,36.459459,36.270270,36.405405,53.459459,18.054054,21.945946,29.081081,35.432432,35.459459
4,36.355556,36.266667,36.288889,36.288889,36.266667,36.311111,36.355556,36.333333,36.355556,53.466667,18.044444,22.000000,29.088889,35.777778,35.822222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6307,36.600000,36.577778,36.577778,36.555556,36.533333,36.511111,36.577778,36.622222,36.555556,53.466667,18.044444,22.088889,30.000000,35.911111,35.888889
6308,36.800000,36.777778,36.800000,36.822222,36.866667,36.844444,36.800000,36.888889,36.800000,53.577778,18.088889,22.666667,30.000000,35.866667,35.866667
6309,36.822222,36.822222,36.822222,36.911111,36.933333,36.866667,36.911111,36.777778,36.822222,53.600000,18.000000,21.955556,30.000000,36.155556,36.133333
6310,36.933333,36.844444,36.866667,36.955556,36.955556,36.955556,36.866667,36.822222,36.888889,53.511111,18.044444,21.911111,30.000000,36.133333,36.133333


In [ ]:
df5

,p1_core16_temp_avg,p1_core17_temp_avg,p1_core18_temp_avg,p1_core19_temp_avg,p1_core20_temp_avg,p1_core21_temp_avg,p1_core2_temp_avg,p1_core3_temp_avg,p1_core4_temp_avg,p1_core5_temp_avg,p1_core6_temp_avg,p1_core7_temp_avg,p1_core8_temp_avg,p1_core9_temp_avg,p1_io_power_avg
0,35.333333,35.303030,35.303030,35.333333,35.363636,35.393939,35.272727,35.272727,35.272727,35.272727,35.333333,35.242424,35.242424,35.242424,50.030303
1,35.300000,35.466667,35.433333,35.400000,35.419355,35.366667,35.531250,35.375000,35.433333,35.468750,35.366667,35.343750,35.225806,35.258065,50.000000
2,35.457143,35.457143,35.457143,35.514286,35.600000,35.600000,35.514286,35.400000,35.342857,35.400000,35.457143,35.514286,35.485714,35.514286,50.028571
3,35.405405,35.540541,35.540541,35.486486,35.513514,35.540541,35.621622,35.594595,35.594595,35.540541,35.540541,35.513514,35.513514,35.486486,50.000000
4,35.800000,35.888889,35.911111,35.866667,35.844444,35.844444,35.755556,35.822222,35.800000,35.777778,35.822222,35.800000,35.800000,35.800000,49.955556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6307,35.822222,35.688889,35.688889,35.755556,35.800000,35.866667,35.822222,35.888889,35.866667,35.911111,35.822222,35.777778,35.866667,35.933333,49.822222
6308,35.911111,35.977778,35.800000,35.777778,36.044444,35.955556,35.866667,35.955556,35.977778,35.911111,35.911111,35.955556,35.844444,35.977778,49.977778
6309,36.022222,36.177778,36.155556,36.044444,36.088889,36.155556,36.066667,36.022222,36.066667,36.088889,36.155556,36.044444,35.977778,36.111111,49.888889
6310,36.155556,36.244444,36.222222,36.355556,36.200000,36.088889,36.155556,36.133333,36.222222,36.155556,36.244444,36.377778,36.177778,36.177778,49.800000


In [ ]:
df6

,p1_mem_power_avg,p1_power_avg,p1_vdd_temp_avg,pcie_avg,ps0_input_power_avg,ps0_input_voltag_avg,ps0_output_curre_avg,ps0_output_volta_avg,ps1_input_power_avg,ps1_input_voltag_avg,ps1_output_curre_avg,ps1_output_volta_avg,total_power_avg,timestamp,value
0,18.000000,24.000000,29.000000,36.090909,230.909091,230.121212,14.969697,12.399999,279.090909,231.272727,17.818182,12.399999,400.000000,2020-03-09 12:00:00+00:00,NaN
1,18.000000,24.000000,29.000000,36.145165,231.612903,230.000000,14.967742,12.399998,279.354839,230.500000,18.166667,12.399998,400.000000,2020-03-09 12:15:00+00:00,NaN
2,18.000000,24.000000,29.000000,36.300003,229.142857,229.885714,15.371429,12.399999,278.571429,230.000000,17.685714,12.399999,400.000000,2020-03-09 12:30:00+00:00,NaN
3,18.000000,24.000000,29.000000,36.300006,231.351351,229.945946,15.000000,12.399999,278.918919,230.000000,17.135135,12.399999,400.000000,2020-03-09 12:45:00+00:00,NaN
4,18.000000,24.000000,29.022222,36.300005,231.555556,228.755556,14.911111,12.400001,277.333333,229.822222,17.888889,12.400001,400.000000,2020-03-09 13:00:00+00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6307,18.000000,25.422222,29.000000,36.593330,229.333333,230.088889,15.133333,12.400001,287.111111,230.888889,18.311111,12.400001,404.888889,2020-06-11 14:45:00+00:00,2
6308,18.044444,25.688889,29.000000,36.599997,230.666667,230.977778,14.866667,12.400001,288.000000,232.000000,18.466667,12.400001,404.888889,2020-06-11 15:00:00+00:00,2
6309,18.044444,24.622222,29.022222,36.599997,230.444444,231.511111,14.888889,12.400001,287.555556,232.000000,18.377778,12.400001,403.111111,2020-06-11 15:15:00+00:00,2
6310,18.000000,24.933333,29.000000,36.599997,231.111111,231.822222,14.777778,12.400001,286.444444,232.000000,18.466667,12.400001,402.222222,2020-06-11 15:30:00+00:00,2


## Create data

### one node

In [ ]:
# def generate_normal_data(rows, start_time, num_nodes, interval=5): ### lets add number of cores and the others to thiss too make it more usable :}
#   timestamps = [start_time + timedelta(minutes=interval * i) for i in range(rows]

#   data = {"timestamp": timestamps}

#   for node in range(num_nodes):
#     ambient_temp = np.random.normal(22, 2, rows)
#     dimm_temp = {i: np.random.normal(35, 5, rows) for i in range(16)}
#     ps_power = {i: np.random.normal(200, 20, rows) for i in range(2)}
#     core_temp = {i: {j: np.random.normal(45, 5, rows) for j in range(24)} for i in range(2)}
#     gpu_core_temp = {i: np.random.normal(60, 10, rows) for i in range(5)}
#     gpu_mem_temp = {i: np.random.normal(65, 10, rows) for i in range(5)}

#     # Add node-specific data
#     data[f"node{node}_ambient_avg"] = ambient_temp ### one node temp
#     data[f"node{node}_ambient_std"] = np.std(ambient_temp)
#     data[f"node{node}_ambient_min"] = np.min(ambient_temp) ### ina ro nemidonam hami ro bezaram ya random bezabam baraye hamshon
#     data[f"node{node}_ambient_max"] = np.max(ambient_temp)

#     for i in range(16): # count of temp for dimm
#       data[f"node{node}_dimm{i}_temp_avg"] = dimm_temp[i]
#       data[f"node{node}_dimm{i}_temp_std"] = np.std(dimm_temp[i])
#       data[f"node{node}_dimm{i}_temp_min"] = np.min(dimm_temp[i])
#       data[f"node{node}_dimm{i}_temp_max"] = np.max(dimm_temp[i])

#     for i in range(2): # count of power suply  has input and output i should chyange it and correct it
#       data[f"node{node}_ps{i}_input_power_avg"] = ps_power[i]
#       data[f"node{node}_ps{i}_input_power_std"] = np.std(ps_power[i])
#       data[f"node{node}_ps{i}_input_power_min"] = np.min(ps_power[i])
#       data[f"node{node}_ps{i}_input_power_max"] = np.max(ps_power[i])

#     for i in range(2): # count of node core temp
#       for j in range(24):
#         data[f"node{node}_p{i}_core{j}_temp_avg"] = core_temp[i][j]
#         data[f"node{node}_p{i}_core{j}_temp_std"] = np.std(core_temp[i][j])
#         data[f"node{node}_p{i}_core{j}_temp_min"] = np.min(core_temp[i][j])
#         data[f"node{node}_p{i}_core{j}_temp_max"] = np.max(core_temp[i][j])

#     for i in range(5): ### count of core and memory for gpu
#       data[f"node{node}_gpu{i}_core_temp_avg"] = gpu_core_temp[i]
#       data[f"node{node}_gpu{i}_core_temp_std"] = np.std(gpu_core_temp[i])
#       data[f"node{node}_gpu{i}_core_temp_min"] = np.min(gpu_core_temp[i])
#       data[f"node{node}_gpu{i}_core_temp_max"] = np.max(gpu_core_temp[i])

#       data[f"node{node}_gpu{i}_mem_temp_avg"] = gpu_mem_temp[i]
#       data[f"node{node}_gpu{i}_mem_temp_std"] = np.std(gpu_mem_temp[i])
#       data[f"node{node}_gpu{i}_mem_temp_min"] = np.min(gpu_mem_temp[i])
#       data[f"node{node}_gpu{i}_mem_temp_max"] = np.max(gpu_mem_temp[i])

#   return pd.DataFrame(data)


############################### not correct
  # Function to generate normal data for multiple nodes
def generate_normal_data(rows, start_time, num_nodes, interval=5):
  timestamps = [start_time + timedelta(minutes=interval * i) for i in range(rows)]

  data = {"timestamp": timestamps}

  for node in range(num_nodes):
    ambient_temp = np.random.normal(22, 2, rows)
    dimm_temp = {i: np.random.normal(35, 5, rows) for i in range(16)}
    ps_input_power = {i: np.random.normal(200, 20, rows) for i in range(2)}
    ps_output_power = {i: np.random.normal(180, 18, rows) for i in range(2)}
    core_temp = {i: {j: np.random.normal(45, 5, rows) for j in range(24)} for i in range(2)}
    gpu_core_temp = {i: np.random.normal(60, 10, rows) for i in range(5)}
    gpu_mem_temp = {i: np.random.normal(65, 10, rows) for i in range(5)}
    fan_speed = {i: np.random.normal(1500, 100, rows) for i in range(4)}
    fan_disk_power = np.random.normal(50, 5, rows)
    p_io_power = {i: np.random.normal(30, 3, rows) for i in range(2)}
    p_mem_power = {i: np.random.normal(25, 2.5, rows) for i in range(2)}
    p_power = {i: np.random.normal(55, 5.5, rows) for i in range(2)}
    p_vdd_temp = {i: np.random.normal(40, 4, rows) for i in range(2)}

    # Add node-specific data
    data[f"node{node}_ambient_avg"] = ambient_temp
    data[f"node{node}_ambient_std"] = np.random.normal(np.std(ambient_temp), 0.5)
    data[f"node{node}_ambient_min"] = np.random.normal(np.min(ambient_temp), 0.5)
    data[f"node{node}_ambient_max"] = np.random.normal(np.max(ambient_temp), 0.5)

    for i in range(16):
      data[f"node{node}_dimm{i}_temp_avg"] = dimm_temp[i]
      data[f"node{node}_dimm{i}_temp_std"] = np.random.normal(np.std(dimm_temp[i]), 0.5)
      data[f"node{node}_dimm{i}_temp_min"] = np.random.normal(np.min(dimm_temp[i]), 0.5)
      data[f"node{node}_dimm{i}_temp_max"] = np.random.normal(np.max(dimm_temp[i]), 0.5)

    for i in range(2):
      data[f"node{node}_ps{i}_input_power_avg"] = ps_input_power[i]
      data[f"node{node}_ps{i}_input_power_std"] = np.random.normal(np.std(ps_input_power[i]), 0.5)
      data[f"node{node}_ps{i}_input_power_min"] = np.random.normal(np.min(ps_input_power[i]), 0.5)
      data[f"node{node}_ps{i}_input_power_max"] = np.random.normal(np.max(ps_input_power[i]), 0.5)

      data[f"node{node}_ps{i}_output_power_avg"] = ps_output_power[i]
      data[f"node{node}_ps{i}_output_power_std"] = np.random.normal(np.std(ps_output_power[i]), 0.5)
      data[f"node{node}_ps{i}_output_power_min"] = np.random.normal(np.min(ps_output_power[i]), 0.5)
      data[f"node{node}_ps{i}_output_power_max"] = np.random.normal(np.max(ps_output_power[i]), 0.5)

      data[f"node{node}_ps{i}_total_power_avg"] = ps_input_power[i] - ps_output_power[i]
      data[f"node{node}_ps{i}_total_power_std"] = np.random.normal(np.std(ps_input_power[i] - ps_output_power[i]), 0.5)
      data[f"node{node}_ps{i}_total_power_min"] = np.random.normal(np.min(ps_input_power[i] - ps_output_power[i]), 0.5)
      data[f"node{node}_ps{i}_total_power_max"] = np.random.normal(np.max(ps_input_power[i] - ps_output_power[i]), 0.5)

    for i in range(2):
      for j in range(24):
        data[f"node{node}_p{i}_core{j}_temp_avg"] = core_temp[i][j]
        data[f"node{node}_p{i}_core{j}_temp_std"] = np.random.normal(np.std(core_temp[i][j]), 0.5)
        data[f"node{node}_p{i}_core{j}_temp_min"] = np.random.normal(np.min(core_temp[i][j]), 0.5)
        data[f"node{node}_p{i}_core{j}_temp_max"] = np.random.normal(np.max(core_temp[i][j]), 0.5)

    for i in range(5):
      data[f"node{node}_gpu{i}_core_temp_avg"] = gpu_core_temp[i]
      data[f"node{node}_gpu{i}_core_temp_std"] = np.random.normal(np.std(gpu_core_temp[i]), 0.5)
      data[f"node{node}_gpu{i}_core_temp_min"] = np.random.normal(np.min(gpu_core_temp[i]), 0.5)
      data[f"node{node}_gpu{i}_core_temp_max"] = np.random.normal(np.max(gpu_core_temp[i]), 0.5)

      data[f"node{node}_gpu{i}_mem_temp_avg"] = gpu_mem_temp[i]
      data[f"node{node}_gpu{i}_mem_temp_std"] = np.random.normal(np.std(gpu_mem_temp[i]), 0.5)
      data[f"node{node}_gpu{i}_mem_temp_min"] = np.random.normal(np.min(gpu_mem_temp[i]), 0.5)
      data[f"node{node}_gpu{i}_mem_temp_max"] = np.random.normal(np.max(gpu_mem_temp[i]), 0.5)

    for i in range(4):
      data[f"node{node}_fan{i}_speed_avg"] = fan_speed[i]
      data[f"node{node}_fan{i}_speed_std"] = np.random.normal(np.std(fan_speed[i]), 0.5)
      data[f"node{node}_fan{i}_speed_min"] = np.random.normal(np.min(fan_speed[i]), 0.5)
      data[f"node{node}_fan{i}_speed_max"] = np.random.normal(np.max(fan_speed[i]), 0.5)

    data[f"node{node}_fan_disk_power_avg"] = fan_disk_power
    data[f"node{node}_fan_disk_power_std"] = np.random.normal(np.std(fan_disk_power), 0.5)
    data[f"node{node}_fan_disk_power_min"] = np.random.normal(np.min(fan_disk_power), 0.5)
    data[f"node{node}_fan_disk_power_max"] = np.random.normal(np.max(fan_disk_power), 0.5)

    for i in range(2):
      data[f"node{node}_p{i}_io_power_avg"] = p_io_power[i]
      data[f"node{node}_p{i}_io_power_std"] = np.random.normal(np.std(p_io_power[i]), 0.5)
      data[f"node{node}_p{i}_io_power_min"] = np.random.normal(np.min(p_io_power[i]), 0.5)
      data[f"node{node}_p{i}_io_power_max"] = np.random.normal(np.max(p_io_power[i]), 0.5)

      data[f"node{node}_p{i}_mem_power_avg"] = p_mem_power[i]
      data[f"node{node}_p{i}_mem_power_std"] = np.random.normal(np.std(p_mem_power[i]), 0.5)
      data[f"node{node}_p{i}_mem_power_min"] = np.random.normal(np.min(p_mem_power[i]), 0.5)
      data[f"node{node}_p{i}_mem_power_max"] = np.random.normal(np.max(p_mem_power[i]), 0.5)

      data[f"node{node}_p{i}_power_avg"] = p_power[i]
      data[f"node{node}_p{i}_power_std"] = np.random.normal(np.std(p_power[i]), 0.5)
      data[f"node{node}_p{i}_power_min"] = np.random.normal(np.min(p_power[i]), 0.5)
      data[f"node{node}_p{i}_power_max"] = np.random.normal(np.max(p_power[i]), 0.5)

      data[f"node{node}_p{i}_vdd_temp_avg"] = p_vdd_temp[i]
      data[f"node{node}_p{i}_vdd_temp_std"] = np.random.normal(np.std(p_vdd_temp[i]), 0.5)
      data[f"node{node}_p{i}_vdd_temp_min"] = np.random.normal(np.min(p_vdd_temp[i]), 0.5)
      data[f"node{node}_p{i}_vdd_temp_max"] = np.random.normal(np.max(p_vdd_temp[i]), 0.5)

  return pd.DataFrame(data)

In [ ]:
########not correct

def generate_normal_data(rows, start_time, num_nodes, interval=5):

  timestamps = [start_time + timedelta(minutes=interval * i) for i in range(rows)]

  data = {"timestamp": timestamps}

  for node in range(num_nodes):
    for row in range(rows):
      # Ambient temperature
      ambient_temp = np.random.normal(22, 2)
      data.setdefault(f"node{node}_ambient_avg", []).append(max(ambient_temp, 0))
      data.setdefault(f"node{node}_ambient_std", []).append(max(np.random.normal(0.5), 0))
      data.setdefault(f"node{node}_ambient_min", []).append(max(np.random.normal(ambient_temp - 2, 0.5), 0))
      data.setdefault(f"node{node}_ambient_max", []).append(max(np.random.normal(ambient_temp + 2, 0.5), 0))

        # DIMM temperatures
      for i in range(16):
        dimm_temp = np.random.normal(35, 5)
        data.setdefault(f"node{node}_dimm{i}_temp_avg", []).append(max(dimm_temp, 0))
        data.setdefault(f"node{node}_dimm{i}_temp_std", []).append(max(np.random.normal(0.5), 0))
        data.setdefault(f"node{node}_dimm{i}_temp_min", []).append(max(np.random.normal(dimm_temp - 3, 0.5), 0))
        data.setdefault(f"node{node}_dimm{i}_temp_max", []).append(max(np.random.normal(dimm_temp + 3, 0.5), 0))

        # Power supply metrics
      for i in range(2):
        ps_input_power = np.random.normal(200, 20)
        ps_output_power = np.random.normal(180, 15)
        total_power = ps_input_power - ps_output_power

        data.setdefault(f"node{node}_ps{i}_input_power_avg", []).append(ps_input_power)
        data.setdefault(f"node{node}_ps{i}_input_power_std", []).append(np.random.normal(5))
        data.setdefault(f"node{node}_ps{i}_input_power_min", []).append(np.random.normal(ps_input_power - 10, 5))
        data.setdefault(f"node{node}_ps{i}_input_power_max", []).append(np.random.normal(ps_input_power + 10, 5))

        data.setdefault(f"node{node}_ps{i}_output_power_avg", []).append(ps_output_power)
        data.setdefault(f"node{node}_ps{i}_output_power_std", []).append(np.random.normal(5))
        data.setdefault(f"node{node}_ps{i}_output_power_min", []).append(np.random.normal(ps_output_power - 10, 5))
        data.setdefault(f"node{node}_ps{i}_output_power_max", []).append(np.random.normal(ps_output_power + 10, 5))

        data.setdefault(f"node{node}_ps{i}_total_power_avg", []).append(total_power)
        data.setdefault(f"node{node}_ps{i}_total_power_std", []).append(np.random.normal(5))
        data.setdefault(f"node{node}_ps{i}_total_power_min", []).append(np.random.normal(total_power - 10, 5))
        data.setdefault(f"node{node}_ps{i}_total_power_max", []).append(np.random.normal(total_power + 10, 5))

        # Processor core temperatures
      for i in range(2):
        for j in range(24):
          core_temp = np.random.normal(45, 5)
          data.setdefault(f"node{node}_p{i}_core{j}_temp_avg", []).append(max(core_temp, 0))
          data.setdefault(f"node{node}_p{i}_core{j}_temp_std", []).append(max(np.random.normal(0.5), 0))
          data.setdefault(f"node{node}_p{i}_core{j}_temp_min", []).append(max(np.random.normal(core_temp - 3, 0.5), 0))
          data.setdefault(f"node{node}_p{i}_core{j}_temp_max", []).append(max(np.random.normal(core_temp + 3, 0.5), 0))

          # Processor power metrics
        p_io_power = np.random.normal(50, 10)
        p_mem_power = np.random.normal(60, 15)
        p_power = np.random.normal(100, 25)
        p_vdd_temp = np.random.normal(70, 10)

        data.setdefault(f"node{node}_p{i}_io_power_avg", []).append(p_io_power)
        data.setdefault(f"node{node}_p{i}_io_power_std", []).append(np.random.normal(0.5))
        data.setdefault(f"node{node}_p{i}_io_power_min", []).append(np.random.normal(p_io_power - 5, 0.5))
        data.setdefault(f"node{node}_p{i}_io_power_max", []).append(np.random.normal(p_io_power + 5, 0.5))

        data.setdefault(f"node{node}_p{i}_mem_power_avg", []).append(p_mem_power)
        data.setdefault(f"node{node}_p{i}_mem_power_std", []).append(np.random.normal(0.5))
        data.setdefault(f"node{node}_p{i}_mem_power_min", []).append(np.random.normal(p_mem_power - 7.5, 0.5))
        data.setdefault(f"node{node}_p{i}_mem_power_max", []).append(np.random.normal(p_mem_power + 7.5, 0.5))

        data.setdefault(f"node{node}_p{i}_power_avg", []).append(p_power)
        data.setdefault(f"node{node}_p{i}_power_std", []).append(np.random.normal(0.5))
        data.setdefault(f"node{node}_p{i}_power_min", []).append(np.random.normal(p_power - 12.5, 0.5))
        data.setdefault(f"node{node}_p{i}_power_max", []).append(np.random.normal(p_power + 12.5, 0.5))

        data.setdefault(f"node{node}_p{i}_vdd_temp_avg", []).append(max(p_vdd_temp, 0))
        data.setdefault(f"node{node}_p{i}_vdd_temp_std", []).append(max(np.random.normal(0.5), 0))
        data.setdefault(f"node{node}_p{i}_vdd_temp_min", []).append(max(np.random.normal(p_vdd_temp - 5, 0.5), 0))
        data.setdefault(f"node{node}_p{i}_vdd_temp_max", []).append(max(np.random.normal(p_vdd_temp + 5, 0.5), 0))

      # GPU temperatures
      for i in range(5):
        gpu_core_temp = np.random.normal(60, 10)
        gpu_mem_temp = np.random.normal(65, 10)
        data.setdefault(f"node{node}_gpu{i}_core_temp_avg", []).append(max(gpu_core_temp, 0))
        data.setdefault(f"node{node}_gpu{i}_core_temp_std", []).append(max(np.random.normal(0.5), 0))
        data.setdefault(f"node{node}_gpu{i}_core_temp_min", []).append(max(np.random.normal(gpu_core_temp - 5, 0.5), 0))
        data.setdefault(f"node{node}_gpu{i}_core_temp_max", []).append(max(np.random.normal(gpu_core_temp + 5, 0.5), 0))

        data.setdefault(f"node{node}_gpu{i}_mem_temp_avg", []).append(max(gpu_mem_temp, 0))
        data.setdefault(f"node{node}_gpu{i}_mem_temp_std", []).append(max(np.random.normal(0.5), 0))
        data.setdefault(f"node{node}_gpu{i}_mem_temp_min", []).append(max(np.random.normal(gpu_mem_temp - 5, 0.5), 0))
        data.setdefault(f"node{node}_gpu{i}_mem_temp_max", []).append(max(np.random.normal(gpu_mem_temp + 5, 0.5), 0))

        # Fan speeds
      for i in range(4):
        fan_speed = np.random.normal(1500, 100)
        data.setdefault(f"node{node}_fan{i}_speed_avg", []).append(max(fan_speed, 0))
        data.setdefault(f"node{node}_fan{i}_speed_std", []).append(max(np.random.normal(50), 0))
        data.setdefault(f"node{node}_fan{i}_speed_min", []).append(max(np.random.normal(fan_speed - 50, 50), 0))
        data.setdefault(f"node{node}_fan{i}_speed_max", []).append(max(np.random.normal(fan_speed + 50, 50), 0))

        # Fan disk power
      fan_disk_power = np.random.normal(10, 2)
      data.setdefault(f"node{node}_fan_disk_power_avg", []).append(max(fan_disk_power, 0))
      data.setdefault(f"node{node}_fan_disk_power_std", []).append(max(np.random.normal(1), 0))
      data.setdefault(f"node{node}_fan_disk_power_min", []).append(max(np.random.normal(fan_disk_power - 1, 1), 0))
      data.setdefault(f"node{node}_fan_disk_power_max", []).append(max(np.random.normal(fan_disk_power + 1, 1), 0))

  return pd.DataFrame(data)


In [ ]:
### use this one
def generate_normal_data(rows, start_time, num_nodes,  core_count, dimm_count, ps_count ,gpu_count , fan_count,ps_power , process_count ,interval=5 ):
    timestamps = [start_time + timedelta(minutes=interval * i) for i in range(rows)]

    data = {"timestamp": timestamps}  ### dictionary sangine che tori in code ro behtar konam??
 #### px (poroccessor x bayad ezafe beshe)
    for node in range(num_nodes):
        ambient_temp = np.random.normal(22, 2, rows)
        dimm_temp = {i: np.random.normal(35, 5, rows) for i in range(dimm_count)}  ## 16
        ps_input_power = {i: np.random.normal(ps_power, 20, rows) for i in range(ps_count)} ## 200  bayad havasam bashe ke kamter az baghie power ha nashe
        ps_output_power = {i: np.random.normal(ps_power-100-60-50, 20, rows) for i in range(ps_count)} ## 180 x alan 200
        core_temp = {i: {j: np.random.normal(45, 5, rows) for j in range(core_count)} for i in range(process_count)} ##24 x   ### proceesc i (2) core j (24)
        gpu_core_temp = {i: np.random.normal(60, 10, rows) for i in range(gpu_count)} ## 5 x
        gpu_mem_temp = {i: np.random.normal(65, 10, rows) for i in range(gpu_count)}  ## 5x
        fan_speed = {i: np.random.normal(2000, 100, rows) for i in range(fan_count)} ### for every core 1 fan
        fan_disk_power = np.random.normal(10, 2, rows)
        p_io_power = {i: np.random.normal(50, 10, rows) for i in range(ps_count)} ## 2
        p_mem_power = {i: np.random.normal(60, 15, rows) for i in range(ps_count)} ## 2
        p_power = {i: np.random.normal(100, 25, rows) for i in range(ps_count)}  ###2
        p_vdd_temp = {i: np.random.normal(70, 10, rows) for i in range(ps_count)}

        data[f"node{node}_ambient_avg"] = ambient_temp
        data[f"node{node}_ambient_std"] = np.random.normal(np.std(ambient_temp), 0.5, rows)
        data[f"node{node}_ambient_min"] = np.random.normal(np.min(ambient_temp), 0.5, rows)
        data[f"node{node}_ambient_max"] = np.random.normal(np.max(ambient_temp), 0.5, rows)

        for i in range(dimm_count):
            data[f"node{node}_dimm{i}_temp_avg"] = dimm_temp[i]
            data[f"node{node}_dimm{i}_temp_std"] = np.random.normal(np.std(dimm_temp[i]), 0.5, rows)
            data[f"node{node}_dimm{i}_temp_min"] = np.random.normal(np.min(dimm_temp[i]), 0.5, rows)
            data[f"node{node}_dimm{i}_temp_max"] = np.random.normal(np.max(dimm_temp[i]), 0.5, rows)

        for i in range(ps_count):
            data[f"node{node}_ps{i}_input_power_avg"] = ps_input_power[i]
            data[f"node{node}_ps{i}_input_power_std"] = np.random.normal(np.std(ps_input_power[i]), 0.5, rows)
            data[f"node{node}_ps{i}_input_power_min"] = np.random.normal(np.min(ps_input_power[i]), 0.5, rows)
            data[f"node{node}_ps{i}_input_power_max"] = np.random.normal(np.max(ps_input_power[i]), 0.5, rows)

            data[f"node{node}_ps{i}_output_power_avg"] = ps_output_power[i]
            data[f"node{node}_ps{i}_output_power_std"] = np.random.normal(np.std(ps_output_power[i]), 0.5, rows)
            data[f"node{node}_ps{i}_output_power_min"] = np.random.normal(np.min(ps_output_power[i]), 0.5, rows)
            data[f"node{node}_ps{i}_output_power_max"] = np.random.normal(np.max(ps_output_power[i]), 0.5, rows)

            total_power = ps_input_power[i] - ps_output_power[i]
            data[f"node{node}_ps{i}_total_power_avg"] = total_power
            data[f"node{node}_ps{i}_total_power_std"] = np.random.normal(np.std(total_power), 0.5, rows)
            data[f"node{node}_ps{i}_total_power_min"] = np.random.normal(np.min(total_power), 0.5, rows)
            data[f"node{node}_ps{i}_total_power_max"] = np.random.normal(np.max(total_power), 0.5, rows)

        for i in range(process_count):
            for j in range(core_count):
                data[f"node{node}_p{i}_core{j}_temp_avg"] = core_temp[i][j]
                data[f"node{node}_p{i}_core{j}_temp_std"] = np.random.normal(np.std(core_temp[i][j]), 0.5, rows)
                data[f"node{node}_p{i}_core{j}_temp_min"] = np.random.normal(np.min(core_temp[i][j]), 0.5, rows)
                data[f"node{node}_p{i}_core{j}_temp_max"] = np.random.normal(np.max(core_temp[i][j]), 0.5, rows)

        for i in range(gpu_count):
            data[f"node{node}_gpu{i}_core_temp_avg"] = gpu_core_temp[i]
            data[f"node{node}_gpu{i}_core_temp_std"] = np.random.normal(np.std(gpu_core_temp[i]), 0.5, rows)
            data[f"node{node}_gpu{i}_core_temp_min"] = np.random.normal(np.min(gpu_core_temp[i]), 0.5, rows)
            data[f"node{node}_gpu{i}_core_temp_max"] = np.random.normal(np.max(gpu_core_temp[i]), 0.5, rows)

            data[f"node{node}_gpu{i}_mem_temp_avg"] = gpu_mem_temp[i]
            data[f"node{node}_gpu{i}_mem_temp_std"] = np.random.normal(np.std(gpu_mem_temp[i]), 0.5, rows)
            data[f"node{node}_gpu{i}_mem_temp_min"] = np.random.normal(np.min(gpu_mem_temp[i]), 0.5, rows)
            data[f"node{node}_gpu{i}_mem_temp_max"] = np.random.normal(np.max(gpu_mem_temp[i]), 0.5, rows)

        for i in range(fan_count):
            data[f"node{node}_fan{i}_speed_avg"] = fan_speed[i]
            data[f"node{node}_fan{i}_speed_std"] = np.random.normal(np.std(fan_speed[i]), 0.5, rows)
            data[f"node{node}_fan{i}_speed_min"] = np.random.normal(np.min(fan_speed[i]), 0.5, rows)
            data[f"node{node}_fan{i}_speed_max"] = np.random.normal(np.max(fan_speed[i]), 0.5, rows)

        data[f"node{node}_fan_disk_power_avg"] = fan_disk_power
        data[f"node{node}_fan_disk_power_std"] = np.random.normal(np.std(fan_disk_power), 0.5, rows)
        data[f"node{node}_fan_disk_power_min"] = np.random.normal(np.min(fan_disk_power), 0.5, rows)
        data[f"node{node}_fan_disk_power_max"] = np.random.normal(np.max(fan_disk_power), 0.5, rows)

        for i in range(ps_count):
            data[f"node{node}_p{i}_io_power_avg"] = p_io_power[i]
            data[f"node{node}_p{i}_io_power_std"] = np.random.normal(np.std(p_io_power[i]), 0.5, rows)
            data[f"node{node}_p{i}_io_power_min"] = np.random.normal(np.min(p_io_power[i]), 0.5, rows)
            data[f"node{node}_p{i}_io_power_max"] = np.random.normal(np.max(p_io_power[i]), 0.5, rows)

            data[f"node{node}_p{i}_mem_power_avg"] = p_mem_power[i]
            data[f"node{node}_p{i}_mem_power_std"] = np.random.normal(np.std(p_mem_power[i]), 0.5, rows)
            data[f"node{node}_p{i}_mem_power_min"] = np.random.normal(np.min(p_mem_power[i]), 0.5, rows)
            data[f"node{node}_p{i}_mem_power_max"] = np.random.normal(np.max(p_mem_power[i]), 0.5, rows)

            data[f"node{node}_p{i}_power_avg"] = p_power[i]
            data[f"node{node}_p{i}_power_std"] = np.random.normal(np.std(p_power[i]), 0.5, rows)
            data[f"node{node}_p{i}_power_min"] = np.random.normal(np.min(p_power[i]), 0.5, rows)
            data[f"node{node}_p{i}_power_max"] = np.random.normal(np.max(p_power[i]), 0.5, rows)

            data[f"node{node}_p{i}_vdd_temp_avg"] = p_vdd_temp[i]
            data[f"node{node}_p{i}_vdd_temp_std"] = np.random.normal(np.std(p_vdd_temp[i]), 0.5, rows)
            data[f"node{node}_p{i}_vdd_temp_min"] = np.random.normal(np.min(p_vdd_temp[i]), 0.5, rows)
            data[f"node{node}_p{i}_vdd_temp_max"] = np.random.normal(np.max(p_vdd_temp[i]), 0.5, rows)  ### ino check konam ke min , max az khodesh bishtar nashe ya kamter
    return data
    # df =pd.DataFrame.from_dict(data)
    # print(type(data['node0_ambient_std']))
    # df = pd.DataFrame(data)
    # df.columns = pd.MultiIndex.from_tuples(df.columns)
    # return df


In [ ]:
start_time = datetime(2024, 1, 1)
rows = 10
num_nodes = 1
interval = 15  # in minutes
core_count=2
dimm_count=4
ps_count=2
gpu_count=2
fan_count=2
ps_power=250
process_count=2
# df = generate_normal_data(rows, start_time, num_nodes, interval)
# print(df["node0_ambient_std"]) generate_normal_data(rows, start_time, num_nodes,  core_count, dimm_count, ps_count ,gpu_count , fan_count,ps_power , process_count ,interval=5 )
data = generate_normal_data(rows, start_time, num_nodes, core_count, dimm_count, ps_count ,gpu_count , fan_count,ps_power , process_count, interval)
df=pd.DataFrame()
for key,value in data.items():
  if type(value) != type([]):
    df[key]= value.tolist()
  else:
    df[key]= value


In [ ]:
df

### ye moshkel daram ke node ziad beshe in kheili column migigre va in khob nist

,timestamp,node0_ambient_avg,node0_ambient_std,node0_ambient_min,node0_ambient_max,node0_dimm0_temp_avg,node0_dimm0_temp_std,node0_dimm0_temp_min,node0_dimm0_temp_max,node0_dimm1_temp_avg,...,node0_p1_mem_power_min,node0_p1_mem_power_max,node0_p1_power_avg,node0_p1_power_std,node0_p1_power_min,node0_p1_power_max,node0_p1_vdd_temp_avg,node0_p1_vdd_temp_std,node0_p1_vdd_temp_min,node0_p1_vdd_temp_max
0,2024-01-01 00:00:00,25.162983,1.042804,18.836647,25.371913,28.315600,5.029033,27.849163,43.243172,33.213442,...,53.610990,81.058459,126.871941,35.558896,47.224076,158.320805,78.514400,11.555025,50.676253,87.210325
1,2024-01-01 00:15:00,20.909959,1.861031,18.144511,24.955574,32.697616,5.079385,28.977484,44.123572,34.955241,...,53.819562,80.756411,120.796301,34.446401,47.148105,158.494271,75.151852,10.360461,50.805050,87.161579
2,2024-01-01 00:30:00,21.332282,1.649281,18.393255,25.339159,31.951531,4.898488,28.651922,43.591550,36.809777,...,53.632050,80.475618,158.333113,35.070076,47.001873,157.902200,51.083268,10.493922,50.772985,88.605407
3,2024-01-01 00:45:00,21.648019,1.804693,18.192962,24.595915,36.147689,4.437134,28.694457,43.674866,23.906044,...,54.936414,81.058526,78.595457,34.778058,47.663142,158.481817,80.134000,11.193234,51.214734,87.794434
4,2024-01-01 01:00:00,21.070140,1.978009,17.491259,24.969046,40.320813,4.707139,27.807260,44.108065,40.160587,...,53.574949,81.308072,125.717681,34.330010,46.109234,157.657510,77.020500,10.993111,51.031314,88.063684
5,2024-01-01 01:15:00,18.457114,1.430421,18.147880,25.670274,40.952055,4.518478,28.884644,43.435202,41.101206,...,54.170267,79.786615,47.038560,34.942522,46.559222,158.443129,87.520112,10.979839,50.993756,87.283957
6,2024-01-01 01:30:00,21.829930,1.267678,18.613571,24.756484,43.930610,5.247586,27.307233,43.648678,34.057194,...,53.823423,79.853250,54.396663,34.562807,47.067383,158.409807,71.927023,11.020698,50.950350,87.661079
7,2024-01-01 01:45:00,21.717814,1.334411,19.125126,25.473258,36.430012,4.173997,28.956155,43.192532,41.563701,...,53.488998,81.293399,136.691871,34.501731,46.803496,158.123935,67.400277,11.273512,51.301257,87.009802
8,2024-01-01 02:00:00,23.031856,0.891184,18.665667,24.849716,39.764595,4.442900,28.300504,43.920560,39.866229,...,53.726007,81.856118,92.109979,35.498931,46.635363,158.039183,51.919655,12.060216,49.918764,86.638806
9,2024-01-01 02:15:00,22.743716,1.219631,18.133111,25.244311,36.874759,4.706963,28.328134,43.812675,43.111839,...,54.796843,79.813477,126.488561,34.842813,48.370054,158.516759,71.165661,11.282273,50.971979,88.321973


In [ ]:
print(df.head())

            timestamp  node0_ambient_avg  node0_ambient_std  \
0 2024-01-01 00:00:00          23.658280           0.648214   
1 2024-01-01 00:15:00          21.246246           0.628456   
2 2024-01-01 00:30:00          20.475114           1.168140   
3 2024-01-01 00:45:00          21.283734           0.590516   
4 2024-01-01 01:00:00          21.757505           1.252101   

   node0_ambient_min  node0_ambient_max  node0_dimm0_temp_avg  \
0          19.681820          23.540093             42.405412   
1          19.414102          23.080799             34.242727   
2          19.936680          23.709810             34.545817   
3          19.478093          24.315231             26.609666   
4          19.606130          24.624103             39.949709   

   node0_dimm0_temp_std  node0_dimm0_temp_min  node0_dimm0_temp_max  \
0              7.023574             20.708758             41.999592   
1              6.807300             20.687595             42.766058   
2              6.

In [ ]:
#### not completed (need to make the rows less or save it in something to better and make limit of size to not get so big and need to think about each node has one csv)
df.to_csv("synthetic_data_with_anomalies.csv", index=False)

In [ ]:
# def inject_anomalies(df, anomaly_types, severity, num_nodes):
#   anomaly_indices = random.sample(range(len(df)), len(df) // 10)  # 10% anomalies

#   for idx in anomaly_indices:
#     if "CPU Spike" in anomaly_types:
#       # Adjust processor core temperatures across all nodes
#       for node in range(num_nodes):
#         for j in range(24):
#           df.at[idx, f'node{node}_p0_core{j}_temp_avg'] += np.random.normal(severity["CPU Spike"], 2)
#           df.at[idx, f'node{node}_p1_core{j}_temp_avg'] += np.random.normal(severity["CPU Spike"], 2)

#     if "Memory Spike" in anomaly_types:
#       # Adjust DIMM temperatures across all nodes
#       for node in range(num_nodes):
#         for i in range(16):
#           df.at[idx, f'node{node}_dimm{i}_temp_avg'] += np.random.normal(severity["Memory Spike"], 2)

#     if "Disk I/O Spike" in anomaly_types:
#       # Adjust power supply metrics across all nodes
#       for node in range(num_nodes):
#         for i in range(2):
#           df.at[idx, f'node{node}_ps{i}_input_power_avg'] += np.random.normal(severity["Disk I/O Spike"], 5)

#     if "Network Spike" in anomaly_types:
#       # Adjust GPU temperatures across all nodes
#       for node in range(num_nodes):
#         for i in range(5):
#           df.at[idx, f'node{node}_gpu{i}_core_temp_avg'] += np.random.normal(severity["Network Spike"], 5)
#           df.at[idx, f'node{node}_gpu{i}_mem_temp_avg'] += np.random.normal(severity["Network Spike"], 5)

#     if "Temperature Spike" in anomaly_types:
#       # Adjust ambient temperature across all nodes
#       for node in range(num_nodes):
#         df.at[idx, f'node{node}_ambient_avg'] += np.random.normal(severity["Temperature Spike"], 1)

#   # Recalculate min, max, and std after injecting anomalies
#   for node in range(num_nodes):
#     for col in ['ambient_avg', 'ambient_min', 'ambient_max']:
#       df[f"node{node}_{col}"] = df[f"node{node}_ambient_avg"].rolling(window=len(df), min_periods=1).agg([np.min, np.max, np.std])[col]

#     for i in range(16):
#       df[f"node{node}_dimm{i}_temp_std"] = df[f"node{node}_dimm{i}_temp_avg"].rolling(window=len(df), min_periods=1).std()
#       df[f"node{node}_dimm{i}_temp_min"] = df[f"node{node}_dimm{i}_temp_avg"].rolling(window=len(df), min_periods=1).min()
#       df[f"node{node}_dimm{i}_temp_max"] = df[f"node{node}_dimm{i}_temp_avg"].rolling(window=len(df), min_periods=1).max()

#     for i in range(2):
#       for j in range(24):
#         df[f"node{node}_p{i}_core{j}_temp_std"] = df[f"node{node}_p{i}_core{j}_temp_avg"].rolling(window=len(df), min_periods=1).std()
#         df[f"node{node}_p{i}_core{j}_temp_min"] = df[f"node{node}_p{i}_core{j}_temp_avg"].rolling(window=len(df), min_periods=1).min()
#         df[f"node{node}_p{i}_core{j}_temp_max"] = df[f"node{node}_p{i}_core{j}_temp_avg"].rolling(window=len(df), min_periods=1).max()

#     for i in range(5):
#       for temp_type in ['core_temp', 'mem_temp']:
#         df[f"node{node}_gpu{i}_{temp_type}_std"] = df[f"node{node}_gpu{i}_{temp_type}_avg"].rolling(window=len(df), min_periods=1).std()
#         df[f"node{node}_gpu{i}_{temp_type}_min"] = df[f"node{node}_gpu{i}_{temp_type}_avg"].rolling(window=len(df), min_periods=1).min()
#         df[f"node{node}_gpu{i}_{temp_type}_max"] = df[f"node{node}_gpu{i}_{temp_type}_avg"].rolling(window=len(df), min_periods=1).max()

#   return df

################################################################ not correct
# Function to inject anomalies
def inject_anomalies(df, anomaly_types, severity, num_nodes):
  anomaly_indices = random.sample(range(len(df)), len(df) // 10)  # 10% anomalies

  for idx in anomaly_indices:
    if "CPU Spike" in anomaly_types:
      # Adjust processor core temperatures across all nodes
      for node in range(num_nodes):
        for j in range(24):
          df.at[idx, f'node{node}_p0_core{j}_temp_avg'] += np.random.normal(severity["CPU Spike"], 2)
          df.at[idx, f'node{node}_p1_core{j}_temp_avg'] += np.random.normal(severity["CPU Spike"], 2)

    if "Memory Spike" in anomaly_types:
      # Adjust DIMM temperatures across all nodes
      for node in range(num_nodes):
        for i in range(16):
          df.at[idx, f'node{node}_dimm{i}_temp_avg'] += np.random.normal(severity["Memory Spike"], 2)

    if "Disk I/O Spike" in anomaly_types:
      # Adjust power supply metrics across all nodes
      for node in range(num_nodes):
        for i in range(2):
          df.at[idx, f'node{node}_ps{i}_input_power_avg'] += np.random.normal(severity["Disk I/O Spike"], 5)
          df.at[idx, f'node{node}_ps{i}_output_power_avg'] += np.random.normal(severity["Disk I/O Spike"], 5)
          df.at[idx, f'node{node}_ps{i}_total_power_avg'] = df.at[idx, f'node{node}_ps{i}_input_power_avg'] - df.at[idx, f'node{node}_ps{i}_output_power_avg']

    if "Network Spike" in anomaly_types:
      # Adjust GPU temperatures across all nodes
      for node in range(num_nodes):
        for i in range(5):
          df.at[idx, f'node{node}_gpu{i}_core_temp_avg'] += np.random.normal(severity["Network Spike"], 5)
          df.at[idx, f'node{node}_gpu{i}_mem_temp_avg'] += np.random.normal(severity["Network Spike"], 5)

    if "Temperature Spike" in anomaly_types:
      # Adjust ambient temperature across all nodes
      for node in range(num_nodes):
        df.at[idx, f'node{node}_ambient_avg'] += np.random.normal(severity["Temperature Spike"], 1)

  # Recalculate min, max, and std after injecting anomalies
  for node in range(num_nodes):
    df[f"node{node}_ambient_std"] = df[f"node{node}_ambient_avg"].rolling(window=len(df), min_periods=1).std()
    df[f"node{node}_ambient_min"] = df[f"node{node}_ambient_avg"].rolling(window=len(df), min_periods=1).min()
    df[f"node{node}_ambient_max"] = df[f"node{node}_ambient_avg"].rolling(window=len(df), min_periods=1).max()

    for i in range(16):
      df[f"node{node}_dimm{i}_temp_std"] = df[f"node{node}_dimm{i}_temp_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_dimm{i}_temp_min"] = df[f"node{node}_dimm{i}_temp_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_dimm{i}_temp_max"] = df[f"node{node}_dimm{i}_temp_avg"].rolling(window=len(df), min_periods=1).max()

    for i in range(2):
      df[f"node{node}_ps{i}_input_power_std"] = df[f"node{node}_ps{i}_input_power_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_ps{i}_input_power_min"] = df[f"node{node}_ps{i}_input_power_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_ps{i}_input_power_max"] = df[f"node{node}_ps{i}_input_power_avg"].rolling(window=len(df), min_periods=1).max()

      df[f"node{node}_ps{i}_output_power_std"] = df[f"node{node}_ps{i}_output_power_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_ps{i}_output_power_min"] = df[f"node{node}_ps{i}_output_power_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_ps{i}_output_power_max"] = df[f"node{node}_ps{i}_output_power_avg"].rolling(window=len(df), min_periods=1).max()

      df[f"node{node}_ps{i}_total_power_std"] = df[f"node{node}_ps{i}_total_power_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_ps{i}_total_power_min"] = df[f"node{node}_ps{i}_total_power_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_ps{i}_total_power_max"] = df[f"node{node}_ps{i}_total_power_avg"].rolling(window=len(df), min_periods=1).max()

    for i in range(24):
      df[f"node{node}_p0_core{i}_temp_std"] = df[f"node{node}_p0_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_p0_core{i}_temp_min"] = df[f"node{node}_p0_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_p0_core{i}_temp_max"] = df[f"node{node}_p0_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).max()

      df[f"node{node}_p1_core{i}_temp_std"] = df["node{node}_p1_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_p1_core{i}_temp_min"] = df[f"node{node}_p1_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_p1_core{i}_temp_max"] = df[f"node{node}_p1_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).max()

    for i in range(5):
      df[f"node{node}_gpu{i}_core_temp_std"] = df[f"node{node}_gpu{i}_core_temp_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_gpu{i}_core_temp_min"] = df[f"node{node}_gpu{i}_core_temp_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_gpu{i}_core_temp_max"] = df[f"node{node}_gpu{i}_core_temp_avg"].rolling(window=len(df), min_periods=1).max()

      df[f"node{node}_gpu{i}_mem_temp_std"] = df[f"node{node}_gpu{i}_mem_temp_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_gpu{i}_mem_temp_min"] = df[f"node{node}_gpu{i}_mem_temp_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_gpu{i}_mem_temp_max"] = df[f"node{node}_gpu{i}_mem_temp_avg"].rolling(window=len(df), min_periods=1).max()

    for i in range(4):
      df[f"node{node}_fan{i}_speed_std"] = df[f"node{node}_fan{i}_speed_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_fan{i}_speed_min"] = df[f"node{node}_fan{i}_speed_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_fan{i}_speed_max"] = df[f"node{node}_fan{i}_speed_avg"].rolling(window=len(df), min_periods=1).max()

    df[f"node{node}_fan_disk_power_std"] = df[f"node{node}_fan_disk_power_avg"].rolling(window=len(df), min_periods=1).std()
    df[f"node{node}_fan_disk_power_min"] = df[f"node{node}_fan_disk_power_avg"].rolling(window=len(df), min_periods=1).min()
    df[f"node{node}_fan_disk_power_max"] = df[f"node{node}_fan_disk_power_avg"].rolling(window=len(df), min_periods=1).max()

    for i in range(2):
      df[f"node{node}_p{i}_io_power_std"] = df[f"node{node}_p{i}_io_power_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_p{i}_io_power_min"] = df[f"node{node}_p{i}_io_power_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_p{i}_io_power_max"] = df[f"node{node}_p{i}_io_power_avg"].rolling(window=len(df), min_periods=1).max()

      df[f"node{node}_p{i}_mem_power_std"] = df[f"node{node}_p{i}_mem_power_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_p{i}_mem_power_min"] = df[f"node{node}_p{i}_mem_power_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_p{i}_mem_power_max"] = df[f"node{node}_p{i}_mem_power_avg"].rolling(window=len(df), min_periods=1).max()

      df[f"node{node}_p{i}_power_std"] = df[f"node{node}_p{i}_power_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_p{i}_power_min"] = df[f"node{node}_p{i}_power_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_p{i}_power_max"] = df[f"node{node}_p{i}_power_avg"].rolling(window=len(df), min_periods=1).max()

      df[f"node{node}_p{i}_vdd_temp_std"] = df[f"node{node}_p{i}_vdd_temp_avg"].rolling(window=len(df), min_periods=1).std()
      df[f"node{node}_p{i}_vdd_temp_min"] = df[f"node{node}_p{i}_vdd_temp_avg"].rolling(window=len(df), min_periods=1).min()
      df[f"node{node}_p{i}_vdd_temp_max"] = df[f"node{node}_p{i}_vdd_temp_avg"].rolling(window=len(df), min_periods=1).max()

  return df


In [ ]:
### use this one
def inject_anomalies(df, anomaly_types, severity, num_nodes, core_count, dimm_count, ps_count, gpu_count, fan_count, process_count):
    anomaly_indices = random.sample(range(len(df)), len(df) // 10)  # 10% anomalies  ### len(df)== rows
    for idx in anomaly_indices:
        if "CPU Spike" in anomaly_types:  ## should  change it and add for others to change too
            # Adjust processor core temperatures across all nodes
            for node in range(num_nodes):
                for j in range(core_count):
                    df.at[idx, f'node{node}_p0_core{j}_temp_avg'] += np.random.normal(severity["CPU Spike"], 2)
                    df.at[idx, f'node{node}_p1_core{j}_temp_avg'] += np.random.normal(severity["CPU Spike"], 2)
                    ############################################ az chat gpt beporse ba che chizi bayad behsh ezafeh beshe
                    df.at[idx, f'node{node}_ps{i}_input_power_avg'] += np.random.normal(severity["Disk I/O Spike"], 5)
                    df.at[idx, f'node{node}_ps{i}_output_power_avg'] += np.random.normal(severity["Disk I/O Spike"], 5)
                    df.at[idx, f'node{node}_ps{i}_total_power_avg'] = df.at[idx, f'node{node}_ps{i}_input_power_avg'] - df.at[idx, f'node{node}_ps{i}_output_power_avg']



        if "Memory Spike" in anomaly_types:
            # Adjust DIMM temperatures across all nodes
            for node in range(num_nodes):
                for i in range(dimm_count):
                    df.at[idx, f'node{node}_dimm{i}_temp_avg'] += np.random.normal(severity["Memory Spike"], 2)

        if "Disk I/O Spike" in anomaly_types:
            # Adjust power supply metrics across all nodes
            for node in range(num_nodes):
                for i in range(ps_count):
                    df.at[idx, f'node{node}_ps{i}_input_power_avg'] += np.random.normal(severity["Disk I/O Spike"], 5)
                    df.at[idx, f'node{node}_ps{i}_output_power_avg'] += np.random.normal(severity["Disk I/O Spike"], 5)
                    df.at[idx, f'node{node}_ps{i}_total_power_avg'] = df.at[idx, f'node{node}_ps{i}_input_power_avg'] - df.at[idx, f'node{node}_ps{i}_output_power_avg']

                    df.at[idx, f'node{node}_ambient_avg'] += np.random.normal(severity["Temperature Spike"], 1)

        if "Network Spike" in anomaly_types:
            # Adjust GPU temperatures across all nodes
            for node in range(num_nodes):
                for i in range(gpu_count):
                    df.at[idx, f'node{node}_gpu{i}_core_temp_avg'] += np.random.normal(severity["Network Spike"], 5)
                    df.at[idx, f'node{node}_gpu{i}_mem_temp_avg'] += np.random.normal(severity["Network Spike"], 5)

        if "Temperature Spike" in anomaly_types:
            # Adjust ambient temperature across all nodes
            for node in range(num_nodes):
                df.at[idx, f'node{node}_ambient_avg'] += np.random.normal(severity["Temperature Spike"], 1)

    # Recalculate min, max, and std after injecting anomalies
    for node in range(num_nodes):
        df[f"node{node}_ambient_std"] = df[f"node{node}_ambient_avg"].rolling(window=len(df), min_periods=1).std()
        df[f"node{node}_ambient_min"] = df[f"node{node}_ambient_avg"].rolling(window=len(df), min_periods=1).min()
        df[f"node{node}_ambient_max"] = df[f"node{node}_ambient_avg"].rolling(window=len(df), min_periods=1).max()

        for i in range(dimm_count):
            df[f"node{node}_dimm{i}_temp_std"] = df[f"node{node}_dimm{i}_temp_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_dimm{i}_temp_min"] = df[f"node{node}_dimm{i}_temp_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_dimm{i}_temp_max"] = df[f"node{node}_dimm{i}_temp_avg"].rolling(window=len(df), min_periods=1).max()

        for i in range(ps_count):
            df[f"node{node}_ps{i}_input_power_std"] = df[f"node{node}_ps{i}_input_power_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_ps{i}_input_power_min"] = df[f"node{node}_ps{i}_input_power_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_ps{i}_input_power_max"] = df[f"node{node}_ps{i}_input_power_avg"].rolling(window=len(df), min_periods=1).max()

            df[f"node{node}_ps{i}_output_power_std"] = df[f"node{node}_ps{i}_output_power_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_ps{i}_output_power_min"] = df[f"node{node}_ps{i}_output_power_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_ps{i}_output_power_max"] = df[f"node{node}_ps{i}_output_power_avg"].rolling(window=len(df), min_periods=1).max()

            df[f"node{node}_ps{i}_total_power_std"] = df[f"node{node}_ps{i}_total_power_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_ps{i}_total_power_min"] = df[f"node{node}_ps{i}_total_power_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_ps{i}_total_power_max"] = df[f"node{node}_ps{i}_total_power_avg"].rolling(window=len(df), min_periods=1).max()

        for i in range(core_count):
            df[f"node{node}_p0_core{i}_temp_std"] = df[f"node{node}_p0_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_p0_core{i}_temp_min"] = df[f"node{node}_p0_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_p0_core{i}_temp_max"] = df[f"node{node}_p0_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).max()

            df[f"node{node}_p1_core{i}_temp_std"] = df[f"node{node}_p1_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_p1_core{i}_temp_min"] = df[f"node{node}_p1_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_p1_core{i}_temp_max"] = df[f"node{node}_p1_core{i}_temp_avg"].rolling(window=len(df), min_periods=1).max()

        for i in range(gpu_count):
            df[f"node{node}_gpu{i}_core_temp_std"] = df[f"node{node}_gpu{i}_core_temp_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_gpu{i}_core_temp_min"] = df[f"node{node}_gpu{i}_core_temp_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_gpu{i}_core_temp_max"] = df[f"node{node}_gpu{i}_core_temp_avg"].rolling(window=len(df), min_periods=1).max()

            df[f"node{node}_gpu{i}_mem_temp_std"] = df[f"node{node}_gpu{i}_mem_temp_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_gpu{i}_mem_temp_min"] = df[f"node{node}_gpu{i}_mem_temp_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_gpu{i}_mem_temp_max"] = df[f"node{node}_gpu{i}_mem_temp_avg"].rolling(window=len(df), min_periods=1).max()

        for i in range(fan_count):
            df[f"node{node}_fan{i}_speed_std"] = df[f"node{node}_fan{i}_speed_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_fan{i}_speed_min"] = df[f"node{node}_fan{i}_speed_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_fan{i}_speed_max"] = df[f"node{node}_fan{i}_speed_avg"].rolling(window=len(df), min_periods=1).max()

        df[f"node{node}_fan_disk_power_std"] = df[f"node{node}_fan_disk_power_avg"].rolling(window=len(df), min_periods=1).std()
        df[f"node{node}_fan_disk_power_min"] = df[f"node{node}_fan_disk_power_avg"].rolling(window=len(df), min_periods=1).min()
        df[f"node{node}_fan_disk_power_max"] = df[f"node{node}_fan_disk_power_avg"].rolling(window=len(df), min_periods=1).max()

        for i in range(process_count):
            df[f"node{node}_p{i}_io_power_std"] = df[f"node{node}_p{i}_io_power_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_p{i}_io_power_min"] = df[f"node{node}_p{i}_io_power_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_p{i}_io_power_max"] = df[f"node{node}_p{i}_io_power_avg"].rolling(window=len(df), min_periods=1).max()

            df[f"node{node}_p{i}_mem_power_std"] = df[f"node{node}_p{i}_mem_power_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_p{i}_mem_power_min"] = df[f"node{node}_p{i}_mem_power_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_p{i}_mem_power_max"] = df[f"node{node}_p{i}_mem_power_avg"].rolling(window=len(df), min_periods=1).max()

            df[f"node{node}_p{i}_power_std"] = df[f"node{node}_p{i}_power_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_p{i}_power_min"] = df[f"node{node}_p{i}_power_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_p{i}_power_max"] = df[f"node{node}_p{i}_power_avg"].rolling(window=len(df), min_periods=1).max()

            df[f"node{node}_p{i}_vdd_temp_std"] = df[f"node{node}_p{i}_vdd_temp_avg"].rolling(window=len(df), min_periods=1).std()
            df[f"node{node}_p{i}_vdd_temp_min"] = df[f"node{node}_p{i}_vdd_temp_avg"].rolling(window=len(df), min_periods=1).min()
            df[f"node{node}_p{i}_vdd_temp_max"] = df[f"node{node}_p{i}_vdd_temp_avg"].rolling(window=len(df), min_periods=1).max()

    return df


In [ ]:

# # Configuration
# rows = 1000
# start_time = datetime(2024, 1, 1)
# num_nodes = 10
# interval = 15  # in minutes
# anomaly_types = ["CPU Spike", "Memory Spike", "Disk I/O Spike", "Network Spike", "Temperature Spike"]
# severity = {
#   "CPU Spike": 20,
#   "Memory Spike": 15,
#   "Disk I/O Spike": 50,
#   "Network Spike": 30,
#   "Temperature Spike": 10
# }

# # Generate normal data
# df = generate_normal_data(rows, start_time, num_nodes, interval)

# # Inject anomalies
# df_with_anomalies = inject_anomalies(df, anomaly_types, severity, num_nodes)

# # Display the dataset
# print(df_with_anomalies.head())

# # Save the dataset to a CSV file
# df_with_anomalies.to_csv("synthetic_data_with_anomalies.csv", index=False)

# Generate data
start_time = datetime(2024, 1, 1)
rows = 1000
num_nodes = 3
interval = 15  # in minutes
df = generate_normal_data(rows, start_time, num_nodes, interval)

# Inject anomalies
anomaly_types = ["CPU Spike", "Memory Spike", "Disk I/O Spike", "Network Spike", "Temperature Spike"]
severity = {"CPU Spike": 10, "Memory Spike": 5, "Disk I/O Spike": 50, "Network Spike": 15, "Temperature Spike": 5}
df_anomalies = inject_anomalies(df, anomaly_types, severity, num_nodes)

# Display the dataset
print(df_with_anomalies.head())

# Save the dataset to a CSV file
df_with_anomalies.to_csv("synthetic_data_with_anomalies.csv", index=False)



### network nodes

## Model

# this is reading real data and model

In [ ]:
#The dataset, collected on the Marconi 100 system, has been published and
#described in the Scientific dataset publication: https://www.nature.com/articles/s41597-023-02174-3
#It is available on Zenodo: https://zenodo.org/records/7541722


#This is rack 1 of the dataset
!wget https://zenodo.org/record/7541722/files/1.tar?download=1 -o rack.tar

In [ ]:
!tar -xf /content/1.tar?download=1

In [ ]:
#This is node 36, of rack 1
file_name = '36.parquet'

In [ ]:
DATA = pd.read_parquet(file_name)
DATA.head(5)

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,2020-03-09 12:00:00+00:00,21.636362,0.077138,21.600000,21.799999,29.0,0.0,29.0,29.0,30.0,...,22.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
1,2020-03-09 12:15:00+00:00,21.860000,0.128063,21.600000,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399998,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
2,2020-03-09 12:30:00+00:00,21.885714,0.098975,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
3,2020-03-09 12:45:00+00:00,21.918917,0.098194,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,20.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
4,2020-03-09 13:00:00+00:00,21.875553,0.096967,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.400001,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>


In [ ]:
DATA = DATA[DATA['timestamp'] > '2021-04-01']

In [ ]:
DATA

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
34338,2021-04-02 00:30:00+00:00,24.720001,0.097979,24.600000,24.799999,33.000000,0.000000,33.0,33.0,33.000000,...,35.0,12.320000,0.074833,12.2,12.4,732.000000,128.124939,580.0,880.0,<NA>
34339,2021-04-02 00:45:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
34340,2021-04-02 01:00:00+00:00,24.785712,0.051508,24.600000,24.799999,33.000000,0.000000,33.0,33.0,33.000000,...,35.0,12.357142,0.072843,12.2,12.4,742.857143,101.940359,580.0,920.0,<NA>
34341,2021-04-02 01:15:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
34342,2021-04-02 01:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86643,2022-09-28 20:45:00+00:00,23.195557,0.186137,22.799999,23.400000,31.933333,0.249444,31.0,32.0,31.288889,...,25.0,12.200001,0.000000,12.2,12.2,548.444444,19.545452,520.0,600.0,0
86644,2022-09-28 21:00:00+00:00,23.746669,0.316649,23.200001,24.200001,31.444444,1.202877,30.0,33.0,31.422222,...,61.0,12.200001,0.000000,12.2,12.2,972.000000,426.487462,520.0,1420.0,0
86645,2022-09-28 21:15:00+00:00,23.271110,0.276986,23.000000,23.799999,30.866667,0.884433,30.0,32.0,30.444444,...,60.0,12.200001,0.000000,12.2,12.2,805.333333,365.036376,520.0,1400.0,0
86646,2022-09-28 21:30:00+00:00,23.079999,0.460724,22.600000,24.200001,31.622222,0.484832,31.0,32.0,31.222222,...,25.0,12.200001,0.000000,12.2,12.2,550.222222,24.448485,500.0,600.0,0


In [ ]:
class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # def __init__(self,latent_dimensions , data_shape):   ##########mesle in bezanam
        # super(SimpleAutoencoder, self).__init__()
        # self.latent_dimensions = latent_dimensions
        # self.data_shape = data_shape

        self.lstm1 = torch.nn.LSTM(
            input_size=352,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16
          hidden_size=8,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device) #### in sparse autoencodedr hastesh :}
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x[:, :, :]

In [ ]:
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=352,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x

In [ ]:
  ######################################################################bahse optimizer ha khhoooooooooooooooooooooooooooooooooobe
##simple_autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
# simple_autoencoder.fit(x_train, x_train,
#                 epochs=1,
#                 shuffle=True,
#                 validation_data=(x_test, x_test))